В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

0. Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv("/content/drive/MyDrive/ML_Course/Data/churn_prediction/train.csv", index_col=0)

In [4]:
input_cols = list(df.columns)[2:-1]
target_col = 'Exited'

In [5]:
train_df, val_df = split_train_val(df, target_col=target_col)

In [6]:
train_inputs, train_targets = separate_inputs_targets(train_df, input_cols=input_cols, target_col=target_col)
val_inputs, val_targets = separate_inputs_targets(val_df, input_cols=input_cols, target_col=target_col)

1. В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [7]:
cat_features = train_inputs.select_dtypes(include='object').columns
train_inputs[cat_features] = train_inputs[cat_features].astype('category')
val_inputs[cat_features] = val_inputs[cat_features].astype('category')

In [8]:
train_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12000 entries, 7180 to 9360
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   CreditScore      12000 non-null  float64 
 1   Geography        12000 non-null  category
 2   Gender           12000 non-null  category
 3   Age              12000 non-null  float64 
 4   Tenure           12000 non-null  float64 
 5   Balance          12000 non-null  float64 
 6   NumOfProducts    12000 non-null  float64 
 7   HasCrCard        12000 non-null  float64 
 8   IsActiveMember   12000 non-null  float64 
 9   EstimatedSalary  12000 non-null  float64 
dtypes: category(2), float64(8)
memory usage: 867.4 KB


2. Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

In [9]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc

In [10]:
def auroc_calc(model, inputs, targets):
  y_pred_proba = model.predict_proba(inputs)[:, 1]

  fpr, tpr, thresholds = roc_curve(targets, y_pred_proba)
  roc_auc = auc(fpr, tpr)
  return roc_auc

In [11]:
xgb_clf = XGBClassifier(
    max_depth=3,
    learning_rate=0.2,
    n_estimators=50,
    n_jobs=-1,
    enable_categorical=True,
    use_label_encoder=False,
    missing=np.nan,
    random_state=42
)

xgb_clf.fit(train_inputs, train_targets)

train_auroc = auroc_calc(xgb_clf, train_inputs, train_targets)
val_auroc = auroc_calc(xgb_clf, val_inputs, val_targets)

print(f'AUROC for Train: {train_auroc}')
print(f'AUROC for VAl: {val_auroc}\n')

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:37:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


AUROC for Train: 0.9435649696949132
AUROC for VAl: 0.9373280060360794



Виходячи з отриманих значень AUROC для тренувального та тестувального наборів, модель демонструє доволі гарні показники. Як на мене, все таки є признаки перенавчання, проте різниця між значеннями доволі не суттєва. Отримане значення на валідаційному тесті є більшим за значення, отримане при використанні Decision Tree в попередньому домашньому завданні.

3. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [12]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [13]:
def objective(params):
    clf = XGBClassifier(
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        enable_categorical=True,
        use_label_encoder=False,
        missing=np.nan,
        early_stopping_rounds=20,
        random_state=42
    )

    clf.fit(
        train_inputs,
        train_targets,
        eval_set=[(val_inputs, val_targets)],
        verbose=False
    )


    auroc = auroc_calc(clf, val_inputs, val_targets)

    return {'loss': -auroc, 'status': STATUS_OK}

space = {
    'n_estimators': hp.quniform('n_estimators', 50, 300, 25),
    'learning_rate': hp.uniform('learning_rate', 0.05, 0.1),
    'max_depth': hp.quniform('max_depth', 2, 6, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.7, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.7, 1.0),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 1, 10),
}



trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials, rstate=np.random.default_rng(42))


best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])
best['min_child_weight'] = int(best['min_child_weight'])

print("Найкращі гіперпараметри: ", best)


final_clf = XGBClassifier(
    n_estimators=best['n_estimators'],
    learning_rate=best['learning_rate'],
    max_depth=best['max_depth'],
    min_child_weight=best['min_child_weight'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    gamma=best['gamma'],
    reg_alpha=best['reg_alpha'],
    reg_lambda=best['reg_lambda'],
    enable_categorical=True,
    use_label_encoder=False,
    missing=np.nan,
    random_state=42
)

final_clf.fit(train_inputs, train_targets)


final_auroc_train = auroc_calc(final_clf, train_inputs, train_targets)
final_auroc_val = auroc_calc(final_clf, val_inputs, val_targets)

print(f'Final AUROC for Train: {final_auroc_train:.4f}')
print(f'Final AUROC for Val: {final_auroc_val:.4f}')

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:37:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



  5%|▌         | 1/20 [00:04<01:17,  4.10s/trial, best loss: -0.9361698333219013]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:37:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 10%|█         | 2/20 [00:05<00:41,  2.31s/trial, best loss: -0.9361698333219013]

...

 95%|█████████▌| 19/20 [00:18<00:00,  2.28trial/s, best loss: -0.9374837094450924]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:37:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



100%|██████████| 20/20 [00:19<00:00,  1.05trial/s, best loss: -0.9374837094450924]
Найкращі гіперпараметри:  {'colsample_bytree': 0.8137862204086027, 'gamma': 0.422980000436168, 'learning_rate': 0.06757592226859521, 'max_depth': 3, 'min_child_weight': 10, 'n_estimators': 200, 'reg_alpha': 0.05407787444418399, 'reg_lambda': 7.9424273583625355, 'subsample': 0.9169895211023081}


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:37:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final AUROC for Train: 0.9432
Final AUROC for Val: 0.9374


In [14]:
print("Найкращі гіперпараметри: ", best)

Найкращі гіперпараметри:  {'colsample_bytree': 0.8137862204086027, 'gamma': 0.422980000436168, 'learning_rate': 0.06757592226859521, 'max_depth': 3, 'min_child_weight': 10, 'n_estimators': 200, 'reg_alpha': 0.05407787444418399, 'reg_lambda': 7.9424273583625355, 'subsample': 0.9169895211023081}


In [15]:
test_raw_df = pd.read_csv("/content/drive/MyDrive/ML_Course/Data/churn_prediction/test.csv", index_col=0)

In [16]:
test_inputs = test_raw_df[input_cols].copy()
test_inputs[cat_features] = test_inputs[cat_features].astype('category')

In [17]:
test_inputs['Exited'] = final_clf.predict_proba(test_inputs)[:, 1]
sample_submission = pd.read_csv("/content/drive/MyDrive/ML_Course/Data/churn_prediction/sample_submission.csv", index_col=0)
sample_submission['Exited'] = test_inputs['Exited'].values
sample_submission.head(5)

,Exited
id,
15000,0.083392
15001,0.013735
15002,0.057163
15003,0.537122
15004,0.035394


In [18]:
sample_submission.reset_index().to_csv("/content/drive/MyDrive/ML_Course/Data/churn_prediction/submission_xgb_final_clf.csv", index=False)

4. Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

In [19]:
import lightgbm as lgb
print(lgb.__version__)

4.5.0


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [20]:
cat_feature_indexes = [train_inputs.columns.get_loc(col) for col in cat_features]

In [21]:
lgb_clf = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=4,
    num_leaves=31,
    subsample=0.9,
    colsample_bytree=0.75,
    reg_alpha=0.5,
    reg_lambda=5,
    min_child_samples=20,
    random_state=42,
    n_jobs=-1,
    cat_feature=cat_feature_indexes,
    missing=np.nan
)

lgb_clf.fit(train_inputs, train_targets, eval_set=[(val_inputs, val_targets)])

train_auroc = auroc_calc(lgb_clf, train_inputs, train_targets)
val_auroc = auroc_calc(lgb_clf, val_inputs, val_targets)

print(f'AUROC for Train: {train_auroc}\n')
print(f'AUROC for VAl: {val_auroc}\n')

[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Warning] categorical_feature is set with cat_feature=1,2, categorical_column=1,2 will be ignored. Current value: categorical_feature=1,2
[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000931 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 843
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 10
[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

Модель не погана, демонструє високі значення auroc на обох вибірках, проте рвзниця між auroc для тренувальних та auroc на валідаційних сетах трохи більша, ніж в попередньої моделі. Це означає, що попередня модель трохи краще генералізує.

5. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [22]:
def objective(params):
    clf = lgb.LGBMClassifier(
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        num_leaves=int(params['num_leaves']),
        min_child_samples=int(params['min_child_samples']),
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        min_split_gain=params['min_split_gain'],
        max_bin=int(params['max_bin']),
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        bagging_freq=int(params['bagging_freq']),
        feature_fraction=params['feature_fraction'],
        cat_feature=cat_feature_indexes,
        random_state=42,
        n_jobs=-1

    )

    clf.fit(
        train_inputs,
        train_targets,
        eval_set=[(val_inputs, val_targets)]
        )

    auroc = auroc_calc(clf, val_inputs, val_targets)
    return {'loss': -auroc, 'status': STATUS_OK}


space = {
    'n_estimators': hp.quniform('n_estimators', 300, 400, 25),
    'learning_rate': hp.uniform('learning_rate', 0.03, 0.06),
    'max_depth': hp.quniform('max_depth', 4, 6, 1),
    'num_leaves': hp.quniform('num_leaves', 35, 50, 1),
    'min_child_samples': hp.quniform('min_child_samples', 5, 20, 1),
    'subsample': hp.uniform('subsample', 0.8, 0.9),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 0.8),
    'feature_fraction': hp.uniform('feature_fraction', 0.6, 0.9),
    'min_split_gain': hp.uniform('min_split_gain', 0.01, 0.2),
    'max_bin': hp.quniform('max_bin', 300, 512, 32),
    'reg_alpha': hp.uniform('reg_alpha', 0.3, 0.8),
    'reg_lambda': hp.uniform('reg_lambda', 4, 7),
    'bagging_freq': hp.quniform('bagging_freq', 1, 3, 1)
}



trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials, rstate=np.random.default_rng(42))


best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])
best['num_leaves'] = int(best['num_leaves'])
best['min_child_samples'] = int(best['min_child_samples'])
best['max_bin'] = int(best['max_bin'])
best['bagging_freq'] = int(best['bagging_freq'])


print("Найкращі гіперпараметри: ", best)


final_clf = lgb.LGBMClassifier(
        n_estimators=best['n_estimators'],
        learning_rate=best['learning_rate'],
        max_depth=best['max_depth'],
        num_leaves=best['num_leaves'],
        min_child_samples=best['min_child_samples'],
        subsample=best['subsample'],
        colsample_bytree=best['colsample_bytree'],
        min_split_gain=best['min_split_gain'],
        max_bin=best['max_bin'],
        reg_alpha=best['reg_alpha'],
        reg_lambda=best['reg_lambda'],
        bagging_freq=best['bagging_freq'],
        feature_fraction=best['feature_fraction'],
        cat_feature=cat_feature_indexes,
        random_state=42,
        n_jobs=-1
)


final_clf.fit(train_inputs, train_targets, eval_set=[(val_inputs, val_targets)])

final_auroc_train = auroc_calc(final_clf, train_inputs, train_targets)
final_auroc_val = auroc_calc(final_clf, val_inputs, val_targets)

print(f'Final AUROC for Train: {final_auroc_train:.4f}')
print(f'Final AUROC for Val: {final_auroc_val:.4f}')

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.6257773566213456, colsample_bytree=0.6121669674222713 will be ignored. Current value: feature_fraction=0.6257773566213456
[LightGBM] [Warning] categorical_feature is set with cat_feature=1,2, categorical_column=1,2 will be ignored. Current value: categorical_feature=1,2
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.6257773566213456, colsample_bytree=0.6121669674222713 will be ignored. Current value: feature_fraction=0.6257773566213456
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005359 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1248
[LightGBM] [Info] Number of data

In [23]:
print("Найкращі гіперпараметри: ", best)

Найкращі гіперпараметри:  {'bagging_freq': 3, 'colsample_bytree': 0.7743046086943409, 'feature_fraction': 0.7050719224646391, 'learning_rate': 0.03625886309909266, 'max_bin': 384, 'max_depth': 4, 'min_child_samples': 18, 'min_split_gain': 0.025006823168791832, 'n_estimators': 350, 'num_leaves': 36, 'reg_alpha': 0.7304652409567174, 'reg_lambda': 5.8247474865134095, 'subsample': 0.8300421354790509}


Модель покращила значення auroc на обох сетах, при цьому немає оверфіту. Наразі це найкраща модель з усіх попередніх.

6. Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

Для сабміта на Kaggle обираю модель lgb з тюнінгованими гіперпараметрами. На мій погляд вона найбільш врівноважена і демонструє найкращий показник якості auroc

In [24]:
test_raw_df = pd.read_csv("/content/drive/MyDrive/ML_Course/Data/churn_prediction/test.csv", index_col=0)

In [25]:
test_inputs = test_raw_df[input_cols].copy()
test_inputs[cat_features] = test_inputs[cat_features].astype('category')

In [26]:
test_inputs['Exited'] = final_clf.predict_proba(test_inputs)[:, 1]
sample_submission = pd.read_csv("/content/drive/MyDrive/ML_Course/Data/churn_prediction/sample_submission.csv", index_col=0)
sample_submission['Exited'] = test_inputs['Exited'].values
sample_submission.head(5)

[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.7050719224646391, colsample_bytree=0.7743046086943409 will be ignored. Current value: feature_fraction=0.7050719224646391


,Exited
id,
15000,0.068682
15001,0.011134
15002,0.058980
15003,0.531171
15004,0.028112


In [27]:
sample_submission.reset_index().to_csv("/content/drive/MyDrive/ML_Course/Data/churn_prediction/submission_lgb_final_clf.csv", index=False)